In [ ]:
#!pip install kmodes

In [1]:
# Import libraries

from itertools import product
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

import pymysql
import pymongo
import sqlalchemy
from sqlalchemy import create_engine

import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from kmodes.kmodes import KModes

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
plt.style.use('seaborn-poster')

from scipy import stats
import statsmodels.api as sm

In [2]:
#Load data
from config import mongoConn, remote_db_endpoint, remote_db_name, remote_db_port, remote_db_user, remote_db_pwd

pymysql.install_as_MySQLdb()
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

query = '''

    SELECT 
        *
    FROM
        CryptoSurveyData

'''

users = pd.read_sql(query, conn)

In [3]:
users.set_index('Entry', inplace=True)
people_list = ['User', 'FirstName', 'LastName', 'City', 'State', 'Zip', 'Lat', 'Lng']
attribute_list = ['Age', 'Gender', 'Known', 'Understanding', 'HaveInvested', 'CryptoSafe', 'CryptoConcern', 'MoreRiskCryptoStock']
people = users[people_list]
attributes = users[attribute_list]



In [4]:
model = KModes(n_clusters=3, init='Huang', n_init=10, verbose=1, random_state=4)
clusters = model.fit_predict(attributes)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 1644, cost: 34564.0
Run 1, iteration: 2/100, moves: 28, cost: 34564.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 2704, cost: 34273.0
Run 2, iteration: 2/100, moves: 517, cost: 34273.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 2522, cost: 34405.0
Run 3, iteration: 2/100, moves: 2138, cost: 33818.0
Run 3, iteration: 3/100, moves: 13, cost: 33818.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 3199, cost: 34087.0
Run 4, iteration: 2/100, moves: 1661, cost: 33608.0
Run 4, iteration: 3/100, moves: 331, cost: 33608.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 2318, cost: 34088.0
Init: initializing centroids
Init:

In [5]:
centroids = pd.DataFrame(model.cluster_centroids_, columns=attribute_list)
attributes['cluster'] = clusters
marketing_df = people.merge(attributes, how='inner', left_index=True, right_index=True)

In [6]:
centroids

,Age,Gender,Known,Understanding,HaveInvested,CryptoSafe,CryptoConcern,MoreRiskCryptoStock
0,77,Male,Known,Somewhat Understood,No,Unsure,Lack of Understanding,Crypto
1,51,Female,Unknown,Unknown,No,Unsure,Fraud,Crypto
2,20,Female,Known,Somewhat Understood,Yes,Yes,Fraud,Equal Risk


In [7]:
choice = input('Which group looks most promising?')


Which group looks most promising?2


In [8]:
marketing_targets = marketing_df[marketing_df['cluster']==int(choice)]

In [9]:
marketing_targets

,User,FirstName,LastName,City,State,Zip,Lat,Lng,Age,Gender,Known,Understanding,HaveInvested,CryptoSafe,CryptoConcern,MoreRiskCryptoStock,cluster
Entry,,,,,,,,,,,,,,,,,
4,louise_medina,Louise,Medina,Minot,ME,04258,44.1461,-70.3399,36,Female,Known,Somewhat Understood,No,Yes,Volitility,Equal Risk,2
6,alberto_potts,Alberto,Potts,Dover,TN,37058,36.4942,-87.8787,45,Male,Known,Well Understood,Yes,Yes,Volitility,Equal Risk,2
7,lawrence_hills,Lawrence,Hills,Oklahoma City,OK,73129,35.4312,-97.4913,23,Male,Known,Well Understood,Yes,Yes,Lack of regulation,Stock Market,2
10,lindy_hyde,Lindy,Hyde,Northfield,OH,44067,41.3208,-81.5429,51,Female,Known,Somewhat Understood,Yes,Yes,Lack of Understanding,Crypto,2
14,hildegard_merchant,Hildegard,Merchant,Tolu,KY,42084,NaN,NaN,76,Female,Known,Somewhat Understood,Yes,Yes,Volitility,Equal Risk,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10172,bryon_oneil,Bryon,Oneil,Farmington,KY,42040,36.6000,-88.5186,22,Male,Known,Well Understood,Plan,Yes,Fraud,Equal Risk,2
10176,dovie_hardesty,Dovie,Hardesty,Tortilla Flat,AZ,85290,NaN,NaN,27,Female,Known,Somewhat Understood,Plan,Yes,Lack of regulation,Stock Market,2
10181,emilie_mayers,Emilie,Mayers,El Paso,TX,79958,31.7587,-106.4869,20,Female,Known,Well Understood,Plan,Yes,Volitility,Stock Market,2
